<a href="https://colab.research.google.com/github/natekchua/TensorFlow-Modelling/blob/master/Part3/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [6]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test.csv
Saving heart_train.csv to heart_train.csv


In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [0]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [9]:
!head {train_file_path}

row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12,5.73,23.11,Present,49,25.3,97.2,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1
6,132,6.2,6.47,36.21,Present,62,30.77,14.14,45,0
7,142,4.05,3.38,16.2,Absent,59,20.81,2.62,38,0
8,114,4.08,4.59,14.6,Present,62,23.11,6.72,58,1
9,114,0,3.83,19.4,Present,49,24.86,2.49,29,0


In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

In [11]:

def get_dataset(file, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file,
      batch_size = 50,
      label_name=LABEL_COLUMN,
      na_value ="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

SELECT_COLUMNS = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']
raw_train_data = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_file_path, select_columns=SELECT_COLUMNS)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [13]:
show_batch(raw_train_data)

sbp                 : [120 122 130 140 142 174 146 154 132 124 128 190 128 154 136 116 122 134
 118 112 152 118 168 122 160 136 136 208 142 127 142 134 126 114 152 130
 142 120 146 124 130 144 146 148 130 130 134 144 138 144]
tobacco             : [ 3.7   0.    1.75  0.    0.    0.    3.6   0.    4.05 14.    4.65  4.18
  0.    1.8   1.7   1.03  4.4   1.5   1.25  9.65  0.    0.12 11.4   3.2
 12.    3.46  0.   27.4   7.44  0.    0.    0.    0.    3.6  19.45  0.
  0.28 10.5   4.36  0.    7.28  0.04  0.    5.5   0.56  0.    6.4   0.01
  1.15  0.76]
ldl                 : [ 4.02  5.75  5.46  2.4   4.19  8.46  3.51  4.81  5.15  6.23  3.31  5.05
  2.51  7.13  3.53  2.83  3.18  3.73  4.69  2.29  6.06  1.96  5.08  3.59
  5.73  6.38  4.12  3.12  5.52  2.81  3.72  5.63  5.98  4.16  4.22  2.82
  1.8   2.7   4.31  2.28  3.56  3.38  6.62  7.1   3.3   3.92  8.49  4.41
  5.09 10.53]
adiposity           : [39.66 30.9  34.34 27.89 18.04 35.1  22.67 28.11 26.51 35.96 22.74 24.83
 29.35 34.04 20.13 10.85 1

In [0]:
train_batch, labels_batch = next(iter(raw_train_data)) 
test_batch, labels_batch = next(iter(raw_test_data)) 

In [0]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']
FEATURES = 9

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [18]:
show_batch(packed_train_data)

famhist             : [b'Absent' b'Present' b'Absent' b'Absent' b'Present' b'Absent' b'Absent'
 b'Present' b'Absent' b'Present' b'Absent' b'Absent' b'Absent' b'Present'
 b'Present' b'Present' b'Absent' b'Absent' b'Absent' b'Absent' b'Present'
 b'Absent' b'Absent' b'Absent' b'Absent' b'Present' b'Absent' b'Present'
 b'Absent' b'Absent' b'Present' b'Absent' b'Absent' b'Present' b'Absent'
 b'Absent' b'Absent' b'Present' b'Present' b'Absent' b'Absent' b'Absent'
 b'Present' b'Absent' b'Present' b'Absent' b'Absent' b'Absent' b'Present'
 b'Present']
numeric             : [[138.     0.6    3.81  28.66  54.    28.7    1.46  58.  ]
 [152.     0.     6.06  41.05  51.    40.34   0.    51.  ]
 [180.     0.52   4.23  16.38  55.    22.56  14.77  45.  ]
 [124.     0.     3.23   9.64  59.    22.7    0.    16.  ]
 [136.     8.8    4.26  32.03  52.    31.44  34.35  60.  ]
 [128.     0.     2.98  12.59  65.    20.74   2.06  19.  ]
 [128.     4.65   3.31  22.74  62.    22.95   0.51  48.  ]
 [126.     0.09 

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

In [20]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,137.207595,3.650810,4.776937,25.414405,53.465823,26.137797,16.813468,43.179747
std,18.988306,4.485022,2.024007,7.663108,9.906196,4.231244,24.434489,14.408669
min,101.000000,0.000000,0.980000,7.120000,13.000000,17.750000,0.000000,15.000000
25%,124.000000,0.100000,3.275000,19.860000,47.000000,23.180000,0.510000,32.000000
50%,134.000000,2.020000,4.410000,26.170000,54.000000,25.810000,7.710000,45.000000
75%,148.000000,5.500000,5.900000,31.135000,60.000000,28.450000,23.310000,56.000000
max,208.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [0]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [23]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(8,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7fdfce96d8c8>, mean=array([137.208,   3.651,   4.777,  25.414,  53.466,  26.138,  16.813,
        43.18 ]), std=array([18.988,  4.485,  2.024,  7.663,  9.906,  4.231, 24.434, 14.409])))

In [24]:
train_batch['numeric']

<tf.Tensor: id=266, shape=(50, 8), dtype=float32, numpy=
array([[134.  ,   0.  ,   2.4 ,  21.11,  57.  ,  22.45,   1.37,  18.  ],
       [132.  ,   2.  ,   2.7 ,  21.57,  50.  ,  27.95,   9.26,  37.  ],
       [126.  ,   3.1 ,   2.01,  32.97,  56.  ,  28.63,  26.74,  45.  ],
       [102.  ,   0.4 ,   3.41,  17.22,  56.  ,  23.59,   2.06,  39.  ],
       [164.  ,   5.6 ,   3.17,  30.98,  44.  ,  25.99,  43.2 ,  53.  ],
       [148.  ,  15.  ,   4.98,  36.94,  72.  ,  31.83,  66.27,  41.  ],
       [122.  ,   1.7 ,   5.28,  32.23,  51.  ,  24.08,   0.  ,  54.  ],
       [162.  ,   6.94,   4.55,  33.36,  52.  ,  27.09,  32.06,  43.  ],
       [112.  ,   9.65,   2.29,  17.2 ,  54.  ,  23.53,   0.68,  53.  ],
       [152.  ,  12.18,   4.04,  37.83,  63.  ,  34.57,   4.17,  64.  ],
       [144.  ,   6.75,   5.45,  29.81,  53.  ,  25.62,  26.23,  43.  ],
       [120.  ,   7.5 ,  15.33,  22.  ,  60.  ,  25.31,  34.49,  49.  ],
       [118.  ,   0.  ,   3.67,  12.13,  51.  ,  19.15,   0.6 ,  15

In [26]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

array([[-0.169, -0.814, -1.174, -0.562,  0.357, -0.872, -0.632, -1.748],
       [-0.274, -0.368, -1.026, -0.502, -0.35 ,  0.428, -0.309, -0.429],
       [-0.59 , -0.123, -1.367,  0.986,  0.256,  0.589,  0.406,  0.126],
       [-1.854, -0.725, -0.675, -1.069,  0.256, -0.602, -0.604, -0.29 ],
       [ 1.411,  0.435, -0.794,  0.726, -0.956, -0.035,  1.08 ,  0.682],
       [ 0.568,  2.53 ,  0.1  ,  1.504,  1.871,  1.345,  2.024, -0.151],
       [-0.801, -0.435,  0.249,  0.889, -0.249, -0.486, -0.688,  0.751],
       [ 1.306,  0.733, -0.112,  1.037, -0.148,  0.225,  0.624, -0.012],
       [-1.328,  1.338, -1.229, -1.072,  0.054, -0.616, -0.66 ,  0.682],
       [ 0.779,  1.902, -0.364,  1.62 ,  0.962,  1.993, -0.517,  1.445],
       [ 0.358,  0.691,  0.333,  0.574, -0.047, -0.122,  0.385, -0.012],
       [-0.906,  0.858,  5.214, -0.446,  0.66 , -0.196,  0.723,  0.404],
       [-1.012, -0.814, -0.547, -1.734, -0.249, -1.651, -0.664, -1.956],
       [ 1.2  , -0.145,  2.18 ,  0.138, -1.46 ,  0.

In [0]:
CATEGORIES = {
    'famhist': ['Present', 'Absent']

}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [29]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='famhist', vocabulary_list=('Present', 'Absent'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [31]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(train_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[0. 1.]


In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [34]:
print(preprocessing_layer(train_batch).numpy()[0])

[ 0.     1.    -0.169 -0.814 -1.174 -0.562  0.357 -0.872 -0.632 -1.748]


In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adamax',
    metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [67]:
print("--Train Model--")
model.fit(train_data, epochs=500)

--Train Model--
Epoch 1/500
8/8 [==============================] - 1s 90ms/step - loss: 0.8486 - accuracy: 0.6076
Epoch 2/500
8/8 [==============================] - 0s 8ms/step - loss: 0.7858 - accuracy: 0.6734
Epoch 3/500
8/8 [==============================] - 0s 7ms/step - loss: 0.7466 - accuracy: 0.6734
Epoch 4/500
8/8 [==============================] - 0s 8ms/step - loss: 0.7348 - accuracy: 0.7038
Epoch 5/500
8/8 [==============================] - 0s 8ms/step - loss: 0.7054 - accuracy: 0.7038
Epoch 6/500
8/8 [==============================] - 0s 8ms/step - loss: 0.7018 - accuracy: 0.7215
Epoch 7/500
8/8 [==============================] - 0s 8ms/step - loss: 0.6968 - accuracy: 0.7316
Epoch 8/500
8/8 [==============================] - 0s 8ms/step - loss: 0.6659 - accuracy: 0.7291
Epoch 9/500
8/8 [==============================] - 0s 7ms/step - loss: 0.6600 - accuracy: 0.7468
Epoch 10/500
8/8 [==============================] - 0s 8ms/step - loss: 0.6520 - accuracy: 0.7544
Epoch 11/500

In [68]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

2/2 [==============================] - 0s 82ms/step - loss: 0.0567 - accuracy: 1.0000


Test Loss 0.05667649023234844, Test Accuracy 1.0


In [72]:
predictions = model.predict(test_data)

# Show some results
for prediction, hasCHD in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted CHD Diagnose Rate: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Has CHD" if bool(hasCHD) else "Does not have CHD"))

Predicted CHD Diagnose Rate: 0.01%  | Actual outcome:  Does not have CHD
Predicted CHD Diagnose Rate: 0.07%  | Actual outcome:  Does not have CHD
Predicted CHD Diagnose Rate: 0.34%  | Actual outcome:  Does not have CHD
Predicted CHD Diagnose Rate: 0.01%  | Actual outcome:  Has CHD
Predicted CHD Diagnose Rate: 99.88%  | Actual outcome:  Has CHD
Predicted CHD Diagnose Rate: 0.35%  | Actual outcome:  Does not have CHD
Predicted CHD Diagnose Rate: 16.54%  | Actual outcome:  Does not have CHD
Predicted CHD Diagnose Rate: 98.67%  | Actual outcome:  Has CHD
Predicted CHD Diagnose Rate: 0.00%  | Actual outcome:  Does not have CHD
Predicted CHD Diagnose Rate: 0.70%  | Actual outcome:  Does not have CHD
